In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point

In [2]:
# Path to domain data and LGA
LGA_shapefile_path = "../../data/raw/ABS_LGA/LGA_2024_AUST_GDA94.shp"
crime = pd.read_csv("../../data/manual/CI_LGAa_data.csv")
property_data_path = "../../data/raw/individual_property.csv"

In [3]:
print(crime.head())
print(crime.info())

# Check specific columns for unexpected NaN values
print("Missing values per column:\n", crime.isna().sum())

        Police Region Local Government Area Display year ending  Year  \
0  1 North West Metro                 Total   Year ending March  2024   
1  1 North West Metro                 Yarra   Year ending March  2024   
2  1 North West Metro               Wyndham   Year ending March  2024   
3  1 North West Metro            Whittlesea   Year ending March  2024   
4  1 North West Metro             Nillumbik   Year ending March  2024   

  Incidents Recorded  
0            148,126  
1             10,482  
2             14,447  
3             10,112  
4              1,269  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870 entries, 0 to 869
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Police Region          870 non-null    object
 1   Local Government Area  870 non-null    object
 2   Display year ending    870 non-null    object
 3   Year                   870 non-null    int64 
 4   Incident

In [4]:
crime = crime[crime['Year'] == 2023]

print(crime.head())
print(crime.info())

# Check specific columns for unexpected NaN values
print("Missing values per column:\n", crime.isna().sum())

         Police Region Local Government Area Display year ending  Year  \
15  1 North West Metro                 Total   Year ending March  2023   
16  1 North West Metro                 Yarra   Year ending March  2023   
17  1 North West Metro               Wyndham   Year ending March  2023   
18  1 North West Metro            Whittlesea   Year ending March  2023   
19  1 North West Metro             Nillumbik   Year ending March  2023   

   Incidents Recorded  
15            134,605  
16              9,533  
17             13,792  
18              9,164  
19              1,165  
<class 'pandas.core.frame.DataFrame'>
Index: 87 entries, 15 to 853
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Police Region          87 non-null     object
 1   Local Government Area  87 non-null     object
 2   Display year ending    87 non-null     object
 3   Year                   87 non-null     int64 
 4   I

In [5]:
# Load in datasets
LGA_gdf = gpd.read_file(LGA_shapefile_path)
property_data = pd.read_csv(property_data_path)

In [6]:
property_data

,Address,Cost,Bedrooms,Bathrooms,Closest Gov Secondary School,Gov Secondary Distance,Age under 20,Age 20-39,Age 40-59,Age 60+,...,2023,2026,CBD Distance,Train Distance,Electricity Distance,Hospital Distance,Library Distance,Park Distance,Tourist Attraction Distance,Grocery Distance
0,"8/90 Hambleton Street, Middle Park VIC 3206",410.0,1,1,Albert Park College,1.2 km away,36%,15%,30%,19%,...,16861.0,17038.932086,4.9721,4.2640,3.1100,3.6124,0.4270,0.1886,0.5198,1.6056
1,"3/33 Bevan Street, Albert Park VIC 3206",550.0,1,1,Albert Park College,0.9 km away,28%,18%,28%,26%,...,16861.0,17038.932086,4.0862,4.7513,2.2242,2.0958,0.2448,0.1982,0.2416,0.7198
2,"7/7-9 Faussett Street, Albert Park VIC 3206",490.0,1,1,Albert Park College,0.8 km away,20%,26%,29%,25%,...,16861.0,17038.932086,4.4456,4.5995,2.5835,1.9441,0.0756,0.1749,0.1316,1.0792
3,Albert Park VIC 3206,1280.0,1,1,Albert Park College,0.4 km away,17%,20%,36%,27%,...,16861.0,17038.932086,4.7979,4.9483,2.0891,2.2929,0.5661,0.1422,0.3968,1.4315
4,"214/363 Beaconsfield Parade, Middle Park VIC 3206",350.0,1,1,Albert Park College,1.6 km away,19%,24%,27%,29%,...,16861.0,17038.932086,5.8848,4.2357,4.0227,3.5144,0.6409,0.5429,0.3927,2.2451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6278,"4 Unaipon Street, Leneva VIC 3691",620.0,4,2,Wodonga Senior Secondary College,3.8 km away,NaN,NaN,NaN,NaN,...,13878.0,15913.700388,309.8923,319.6329,11.0071,4.3516,4.9583,2.6430,13.7212,4.6696
6279,"1 Chatham Road, Leneva VIC 3691",600.0,4,2,Wodonga Senior Secondary College,3.8 km away,NaN,NaN,NaN,NaN,...,13878.0,15913.700388,310.1976,319.9381,8.7169,4.6568,5.2636,2.8860,14.0265,4.9748
6280,"8 Roycroft Street, Baranduda VIC 3691",620.0,4,2,Wodonga Senior Secondary College,7.8 km away,NaN,NaN,NaN,NaN,...,13878.0,15913.700388,317.0669,326.8075,3.9450,10.1097,9.7776,2.0333,13.1193,9.9430
6281,"20 WOODBRIDGE STREET, Killara VIC 3691",800.0,4,2,Wodonga Senior Secondary College,5.8 km away,NaN,NaN,NaN,NaN,...,13878.0,15913.700388,316.1258,325.8664,4.2896,9.7304,8.8365,0.9610,8.0988,9.0019


In [7]:
LGA_gdf

,LGA_CODE24,LGA_NAME24,STE_CODE21,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM,LOCI_URI21,geometry
0,10050,Albury,1,New South Wales,AUS,Australia,305.6386,https://linked.data.gov.au/dataset/asgsed3/LGA...,"POLYGON ((146.86565 -36.07293, 146.86511 -36.0..."
1,10180,Armidale,1,New South Wales,AUS,Australia,7809.4406,https://linked.data.gov.au/dataset/asgsed3/LGA...,"POLYGON ((152.38816 -30.52641, 152.38812 -30.5..."
2,10250,Ballina,1,New South Wales,AUS,Australia,484.9692,https://linked.data.gov.au/dataset/asgsed3/LGA...,"MULTIPOLYGON (((153.57106 -28.87382, 153.57105..."
3,10300,Balranald,1,New South Wales,AUS,Australia,21690.7493,https://linked.data.gov.au/dataset/asgsed3/LGA...,"POLYGON ((143.00432 -33.78165, 143.01538 -33.7..."
4,10470,Bathurst,1,New South Wales,AUS,Australia,3817.8645,https://linked.data.gov.au/dataset/asgsed3/LGA...,"POLYGON ((149.84877 -33.52785, 149.84863 -33.5..."
...,...,...,...,...,...,...,...,...,...
561,89799,Migratory - Offshore - Shipping (ACT),8,Australian Capital Territory,AUS,Australia,NaN,https://linked.data.gov.au/dataset/asgsed3/LGA...,None
562,99399,Unincorp. Other Territories,9,Other Territories,AUS,Australia,105.8806,https://linked.data.gov.au/dataset/asgsed3/LGA...,"MULTIPOLYGON (((167.94747 -29.12758, 167.94748..."
563,99499,No usual address (OT),9,Other Territories,AUS,Australia,NaN,https://linked.data.gov.au/dataset/asgsed3/LGA...,None
564,99799,Migratory - Offshore - Shipping (OT),9,Other Territories,AUS,Australia,NaN,https://linked.data.gov.au/dataset/asgsed3/LGA...,None


In [8]:
# Modify property to prepare join with shapefile
property_data['geometry'] = property_data.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
property_gdf = gpd.GeoDataFrame(property_data, geometry='geometry', crs=LGA_gdf.crs)

In [9]:
property_gdf

,Address,Cost,Bedrooms,Bathrooms,Closest Gov Secondary School,Gov Secondary Distance,Age under 20,Age 20-39,Age 40-59,Age 60+,...,2026,CBD Distance,Train Distance,Electricity Distance,Hospital Distance,Library Distance,Park Distance,Tourist Attraction Distance,Grocery Distance,geometry
0,"8/90 Hambleton Street, Middle Park VIC 3206",410.0,1,1,Albert Park College,1.2 km away,36%,15%,30%,19%,...,17038.932086,4.9721,4.2640,3.1100,3.6124,0.4270,0.1886,0.5198,1.6056,POINT (144.96048 -37.84755)
1,"3/33 Bevan Street, Albert Park VIC 3206",550.0,1,1,Albert Park College,0.9 km away,28%,18%,28%,26%,...,17038.932086,4.0862,4.7513,2.2242,2.0958,0.2448,0.1982,0.2416,0.7198,POINT (144.95637 -37.83996)
2,"7/7-9 Faussett Street, Albert Park VIC 3206",490.0,1,1,Albert Park College,0.8 km away,20%,26%,29%,25%,...,17038.932086,4.4456,4.5995,2.5835,1.9441,0.0756,0.1749,0.1316,1.0792,POINT (144.95533 -37.84167)
3,Albert Park VIC 3206,1280.0,1,1,Albert Park College,0.4 km away,17%,20%,36%,27%,...,17038.932086,4.7979,4.9483,2.0891,2.2929,0.5661,0.1422,0.3968,1.4315,POINT (144.95145 -37.84386)
4,"214/363 Beaconsfield Parade, Middle Park VIC 3206",350.0,1,1,Albert Park College,1.6 km away,19%,24%,27%,29%,...,17038.932086,5.8848,4.2357,4.0227,3.5144,0.6409,0.5429,0.3927,2.2451,POINT (144.96131 -37.85404)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6278,"4 Unaipon Street, Leneva VIC 3691",620.0,4,2,Wodonga Senior Secondary College,3.8 km away,NaN,NaN,NaN,NaN,...,15913.700388,309.8923,319.6329,11.0071,4.3516,4.9583,2.6430,13.7212,4.6696,POINT (146.88745 -36.16161)
6279,"1 Chatham Road, Leneva VIC 3691",600.0,4,2,Wodonga Senior Secondary College,3.8 km away,NaN,NaN,NaN,NaN,...,15913.700388,310.1976,319.9381,8.7169,4.6568,5.2636,2.8860,14.0265,4.9748,POINT (146.89306 -36.16054)
6280,"8 Roycroft Street, Baranduda VIC 3691",620.0,4,2,Wodonga Senior Secondary College,7.8 km away,NaN,NaN,NaN,NaN,...,15913.700388,317.0669,326.8075,3.9450,10.1097,9.7776,2.0333,13.1193,9.9430,POINT (146.93763 -36.17952)
6281,"20 WOODBRIDGE STREET, Killara VIC 3691",800.0,4,2,Wodonga Senior Secondary College,5.8 km away,NaN,NaN,NaN,NaN,...,15913.700388,316.1258,325.8664,4.2896,9.7304,8.8365,0.9610,8.0988,9.0019,POINT (146.94251 -36.14017)


In [10]:
# Join
mapped_properties = gpd.sjoin(property_gdf, LGA_gdf, how='left', predicate='within')
mapped_properties['LGA_NAME24'] = mapped_properties['LGA_NAME24'].astype(str)

print(mapped_properties.head())

                                             Address    Cost  Bedrooms  \
0        8/90 Hambleton Street, Middle Park VIC 3206   410.0         1   
1            3/33 Bevan Street, Albert Park VIC 3206   550.0         1   
2        7/7-9 Faussett Street, Albert Park VIC 3206   490.0         1   
3                               Albert Park VIC 3206  1280.0         1   
4  214/363 Beaconsfield Parade, Middle Park VIC 3206   350.0         1   

   Bathrooms Closest Gov Secondary School Gov Secondary Distance Age under 20  \
0          1          Albert Park College            1.2 km away          36%   
1          1          Albert Park College            0.9 km away          28%   
2          1          Albert Park College            0.8 km away          20%   
3          1          Albert Park College            0.4 km away          17%   
4          1          Albert Park College            1.6 km away          19%   

  Age 20-39 Age 40-59 Age 60+  ...                     geometry  ind

In [11]:
crime['Local Government Area'] = crime['Local Government Area'].astype(str)

In [12]:
crime['Local Government Area'] = crime['Local Government Area'].str.strip()
mapped_properties['LGA_NAME24'] = mapped_properties['LGA_NAME24'].str.strip()


In [13]:
# Merge the spatially joined data with property and crime data using the correct column names
merged_data = mapped_properties.merge(
    crime, 
    left_on='LGA_NAME24', 
    right_on='Local Government Area', 
    how='left'
)

In [14]:
merged_data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 6283 entries, 0 to 6282
Data columns (total 40 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   Address                       6283 non-null   object  
 1   Cost                          6283 non-null   float64 
 2   Bedrooms                      6283 non-null   int64   
 3   Bathrooms                     6283 non-null   int64   
 4   Closest Gov Secondary School  5468 non-null   object  
 5   Gov Secondary Distance        5468 non-null   object  
 6   Age under 20                  6210 non-null   object  
 7   Age 20-39                     6210 non-null   object  
 8   Age 40-59                     6210 non-null   object  
 9   Age 60+                       6210 non-null   object  
 10  Postcode                      6283 non-null   int64   
 11  Latitude                      6283 non-null   float64 
 12  Longitude                     6283 non-n

In [15]:
merged_data.head(3)

,Address,Cost,Bedrooms,Bathrooms,Closest Gov Secondary School,Gov Secondary Distance,Age under 20,Age 20-39,Age 40-59,Age 60+,...,STE_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM,LOCI_URI21,Police Region,Local Government Area,Display year ending,Year,Incidents Recorded
0,"8/90 Hambleton Street, Middle Park VIC 3206",410.0,1,1,Albert Park College,1.2 km away,36%,15%,30%,19%,...,Victoria,AUS,Australia,20.6141,https://linked.data.gov.au/dataset/asgsed3/LGA...,3 Southern Metro,Port Phillip,Year ending March,2023.0,"9,809"
1,"3/33 Bevan Street, Albert Park VIC 3206",550.0,1,1,Albert Park College,0.9 km away,28%,18%,28%,26%,...,Victoria,AUS,Australia,20.6141,https://linked.data.gov.au/dataset/asgsed3/LGA...,3 Southern Metro,Port Phillip,Year ending March,2023.0,"9,809"
2,"7/7-9 Faussett Street, Albert Park VIC 3206",490.0,1,1,Albert Park College,0.8 km away,20%,26%,29%,25%,...,Victoria,AUS,Australia,20.6141,https://linked.data.gov.au/dataset/asgsed3/LGA...,3 Southern Metro,Port Phillip,Year ending March,2023.0,"9,809"


In [16]:
selected_columns = [
    'Address', 'Cost', 'Bedrooms', 'Bathrooms',
    'Closest Gov Secondary School', 'Gov Secondary Distance',
    'Age under 20', 'Age 20-39', 'Age 40-59', 'Age 60+', 'Postcode',
    'Latitude', 'Longitude', 'SA2  code', 'income_2020', '2023', '2026',
    'CBD Distance', 'Train Distance', 'Electricity Distance', 
    'Hospital Distance', 'Library Distance', 'Park Distance', 
    'Tourist Attraction Distance', 'Grocery Distance', 
    'Local Government Area', 'Incidents Recorded'
]

# Filter the merged data to keep only the selected columns
filtered_data = merged_data[selected_columns].copy()

In [17]:
filtered_data.head(3)

,Address,Cost,Bedrooms,Bathrooms,Closest Gov Secondary School,Gov Secondary Distance,Age under 20,Age 20-39,Age 40-59,Age 60+,...,CBD Distance,Train Distance,Electricity Distance,Hospital Distance,Library Distance,Park Distance,Tourist Attraction Distance,Grocery Distance,Local Government Area,Incidents Recorded
0,"8/90 Hambleton Street, Middle Park VIC 3206",410.0,1,1,Albert Park College,1.2 km away,36%,15%,30%,19%,...,4.9721,4.2640,3.1100,3.6124,0.4270,0.1886,0.5198,1.6056,Port Phillip,"9,809"
1,"3/33 Bevan Street, Albert Park VIC 3206",550.0,1,1,Albert Park College,0.9 km away,28%,18%,28%,26%,...,4.0862,4.7513,2.2242,2.0958,0.2448,0.1982,0.2416,0.7198,Port Phillip,"9,809"
2,"7/7-9 Faussett Street, Albert Park VIC 3206",490.0,1,1,Albert Park College,0.8 km away,20%,26%,29%,25%,...,4.4456,4.5995,2.5835,1.9441,0.0756,0.1749,0.1316,1.0792,Port Phillip,"9,809"


In [18]:
# Save the updated DataFrame if needed
filtered_data.to_csv("../../data/raw/individual_property_with_crime2023.csv", index=False)